# Stabilizatión of the simulation

In [2]:
## -----Import essential packages

from __future__ import division
import hoomd
import hoomd.md

#-----Define important variables

temp = 9.0
pressure = 0.1
#-----Start from a gsd file

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Select the trayectory of the center of mass of particles

system = hoomd.init.read_gsd("T_CM_9.0_0.1_ramp.gsd",frame = -1)

#-----Define each rigid body type in the local coordinate system of the body.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types=['A']*8,
                positions=[(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Instruct the rigid constraint to create the constituent particles

rigid.create_bodies();

#-----Define the potential energy

nl = hoomd.md.nlist.tree()

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist=nl)
lj.set_params(mode='shift')

#-----Define interaction with species in the binary mixture

lj.pair_coeff.set('M','M', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('A','A', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('M','A', epsilon = 1.0, sigma = 1.0)

#-----Select an standard integrator

hoomd.md.integrate.mode_standard(dt=0.005);

#------Define group

mesogens = hoomd.group.rigid_center();

#----Integrate using NPT

npt = hoomd.md.integrate.npt(group = mesogens, kT = temp, tau = 2.5, tauP = 1.5, P = pressure);

#-----Write output and Run the Simulation

log_file = "T_" + str(temp) + "_" + str(pressure) + "_stabilization.log"
gsd_file = "T_" + str(temp) + "_" + str(pressure) + "_stabilization.gsd" 
meso_gsd_file = "T_CM_" + str(temp) +  "_" + str(pressure) + "_stabilization.gsd"
log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite=True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = mesogens,
               overwrite=True); 

notice(2): Group "all" created containing 1000 particles
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
-----
You are using tree neighbor lists. Please cite the following:
* M P Howard, J A Anderson, A Nikoubashman, S C Glotzer, and A Z
  Panagiotopoulos. "Efficient neighbor list calculation for molecular simulation
  of colloidal systems using graphics processing units", Computer Physics
  Communications 203 (2016) 45--52
-----
notice(2): Group "rigid_center" created containing 1000 particles


In [3]:
hoomd.run(5e5)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9000
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:13 | Step 1826502 / 2326000 | TPS 50.1829 | ETA 02:45:53
Time 00:00:23 | Step 1827057 / 2326000 | TPS 55.3487 | ETA 02:30:14
Time 00:00:34 | Step 1827661 / 2326000 | TPS 60.245 | ETA 02:17:51
Time 00:00:44 | Step 1828235 / 2326000 | TPS 57.2674 | ETA 02:24:51
Time 00:00:54 | Step 1828772 / 2326000 | TPS 53.6956 | ETA 02:34:20
Time 00:01:04 | Step 1829278 / 2326000 | TPS 50.573 | ETA 02:43:41
Time 00:01:14 | Step 1829824 / 2326000 | TPS 54.5763 | ETA 02:31:31
Time 00:01:24 | Step 1830441 / 2326000 | TPS 61.6838 | ETA 02:13:53
Time 00:01:34 | Step 1831055 / 2326000 | TPS 61.3764 | ETA 02:14:24
Time 00:01:44 | Step 1831629 / 2326000 | TPS 57.2498 | ETA 02:23:55
Time 00:01:54 | Step 1832191 / 2326000 | TPS 56.1177 | ETA 02:26:39
Time 00:02:

**ERROR**: nlist: Simulation box is too small! Particles would be interacting with themselves.


RuntimeError: Error updating neighborlist bins